In [96]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [45]:
cid = '6b4356b846d64135bf4bcec341a96b9c'
secret = '0baa0aec07994419887012b77df4a93e'

In [46]:
#Authentication - without user  
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [47]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [58]:
sp.playlist_tracks(playlist_URI)['items'][0]['track'].keys()

dict_keys(['preview_url', 'available_markets', 'explicit', 'type', 'episode', 'track', 'album', 'artists', 'disc_number', 'track_number', 'duration_ms', 'external_ids', 'external_urls', 'href', 'id', 'name', 'popularity', 'uri', 'is_local'])

In [93]:
sp.playlist_tracks(playlist_URI)['items'][0]['track'].keys()

dict_keys(['preview_url', 'available_markets', 'explicit', 'type', 'episode', 'track', 'album', 'artists', 'disc_number', 'track_number', 'duration_ms', 'external_ids', 'external_urls', 'href', 'id', 'name', 'popularity', 'uri', 'is_local'])

In [100]:
artist_uri = sp.playlist_tracks(playlist_URI)['items'][0]['track']['artists'][0]['uri']
#sp.artist(artist_uri)

In [99]:
#working on the world popular song playlist. Later on: if work on different playlists, add a playlist_name column.
songs_dict = {'track_uri':[], 'track_name': [], 'artist_name': [], 'artist_uri': [], 'album_name': [], 'album_uri': [],
            'danceability': [], 'energy': [], 'key': [], 'loudness': [], 'mode': [], 'speechiness': [], 'acousticness': [],
            'instrumentalness': [], 'liveness': [], 'valence': [], 'tempo': [], 'track_href': [], 'analysis_url': [],
            'duration_ms': [], 'time_signature': [], 'artist_pop': [], 'genres': [], 'track_pop': []}
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #track_uri, track_name
    songs_dict['track_uri'].append(track["track"]["uri"])
    songs_dict['track_name'].append(track["track"]["name"])
    
    #artist_uri, artist_name, genres / this would only take information of the first artist, maybe a song could have more than one
    songs_dict['artist_uri'].append(track["track"]["artists"][0]["uri"])
    artist_uri = track["track"]["artists"][0]["uri"]
    songs_dict['artist_name'].append(track["track"]["artists"][0]['name'])
    artist_info = sp.artist(artist_uri)
    songs_dict["genres"].append(artist_info["genres"])

    #album_name, album_uri 
    songs_dict['album_name'].append(track["track"]["album"]["name"])
    songs_dict['album_uri'].append(track["track"]["album"]["uri"])
    
    #audio_features
    audio_features = sp.audio_features(track["track"]["uri"])[0]
    songs_dict["danceability"].append(audio_features["danceability"])
    songs_dict["energy"].append(audio_features["energy"])
    songs_dict["key"].append(audio_features["key"])
    songs_dict["loudness"].append(audio_features["loudness"])
    songs_dict["mode"].append(audio_features["mode"])
    songs_dict["speechiness"].append(audio_features["speechiness"])
    songs_dict["acousticness"].append(audio_features["acousticness"])
    songs_dict["instrumentalness"].append(audio_features["instrumentalness"])
    songs_dict["liveness"].append(audio_features["liveness"])
    songs_dict["valence"].append(audio_features["valence"])
    songs_dict["tempo"].append(audio_features["tempo"])
    songs_dict["track_href"].append(audio_features["track_href"])
    songs_dict["analysis_url"].append(audio_features["analysis_url"])
    songs_dict["duration_ms"].append(audio_features["duration_ms"])
    songs_dict["time_signature"].append(audio_features["time_signature"])

    #artist_pop, track_pop
    songs_dict['artist_pop'].append(artist_info["popularity"])
    songs_dict['track_pop'].append(track["track"]["popularity"])

In [101]:
songs_df = pd.DataFrame.from_dict(songs_dict)

## Data Cleaning

In [112]:
# use to eliminate same song appearing in different playlists, but we are only using 1 playlist, so this function is meaningless
def dedup(df):
    df["artist_song"] = df.apply(lambda row: row['arist_name'] + row['track_name'], axis = 1)
    return df.drop_duplicates('artist_song')

## Features Engineering

### Metadata ( features described the song but not the song itself )

In [ ]:
#genres, track_pop

### Firstly, we need to deal with categorical data, such as genres, key, mode, polar -> One-hot encoding and TF-IDF (weighted OHE)

In [127]:
def ohe(df, col):
    '''''
    Input: 
    - a dataframe containing a column we want to encode: df
    - the name of that column: col
    Output:
    - a new dataframe with encoded information
    '''''
    category_df = pd.get_dummies(df[col])
    feature_name = category_df.columns
    category_df.columns = [str(col) + "|" + str(genre) for genre in feature_name]
    category_df.reset_index(drop = True, inplace = True)
    return category_df